In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autoplan.generator import Generator, Inference
from grammars.pythagorean.program import Program, PythagoreanLabels
from autoplan.dataset import build_synthetic_dataset
from autoplan.models import ProgramClassifier, GrammarInference
from autoplan.trainer import Trainer
from autoplan.token import JavaTokenizer
from tqdm import tqdm_notebook as tqdm
import torch
import pandas as pd

# device = torch.device('cpu')
device = torch.device('cuda:0')

In [60]:
%aimport -torch

In [131]:
program_grammar = Program()
generator = Generator(grammar=program_grammar)
program, choices, choice_options = generator.generate()
print(program)
print(choices)
print(choice_options)


            public class PythagoreanTheorem extends ConsoleProgram {{
                public void run() {{
                    
        print("Enter values to compute the Pythagorean Theorem.");
        int a = readInt("a:");
        int b = readInt("b:");
        int c = Math.sqrt(a * a + b * b);
        print("c=" + c);
        
                }}
            }}
        
[('naming_scheme', 0), ('num_type', 0), ('print_function', 1), ('main_prompt', 0), ('input_prompt', 0), ('uses_method', 1), ('output_prompt', 0)]
{'naming_scheme': [(0.5, 'a'), (0.5, 'x')], 'num_type': [(0.5, 'int'), (0.5, 'double')], 'print_function': [(0.5, 'println'), (0.5, 'print')], 'main_prompt': [(0.699999988079071, 'Enter values to compute the Pythagorean Theorem.'), (0.15000000596046448, 'This program finds the hypotenuse, C, of a triangle with sides A and B.'), (0.15000000596046448, 'This program runs the Pythagorean Theorem. Choose values a and b.')], 'input_prompt': [(0.5555555820465088, ''), (0.22222222

In [121]:
dataset = build_synthetic_dataset(
    N_train=1000, 
    N_val=100, 
    tokenizer=JavaTokenizer(),
    grammar=program_grammar)

In [91]:
infer = Inference(program_grammar, dataset, device)

In [122]:
loader = dataset.loader(dataset.train_dataset)
batch = next(iter(loader))
#infer.infer(batch['program'][:1], batch['program_len'][:1])

In [123]:
batch

{'choices': tensor([[1, 1, 0, 0, 0, 1, 1],
         [1, 1, 1, 0, 3, 0, 0],
         [1, 1, 0, 0, 0, 1, 3],
         [0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 1, 1, 1],
         [0, 1, 1, 0, 2, 0, 1],
         [1, 1, 1, 0, 2, 0, 0],
         [1, 0, 0, 0, 1, 1, 3],
         [1, 1, 1, 0, 1, 1, 1],
         [1, 1, 1, 0, 1, 0, 1],
         [1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 2, 1, 0, 0],
         [1, 1, 1, 2, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0, 3],
         [0, 1, 1, 0, 0, 1, 0],
         [0, 1, 1, 0, 3, 1, 1],
         [1, 0, 1, 0, 2, 0, 0],
         [1, 1, 0, 0, 1, 0, 0],
         [1, 1, 0, 1, 0, 1, 0],
         [0, 0, 1, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 1, 0],
         [0, 1, 0, 0, 0, 1, 0],
         [1, 1, 0, 0, 0, 1, 0],
         [0, 1, 1, 0, 0, 0, 0],
         [0, 0, 1, 2, 0, 0, 0],
         [0, 0, 1, 0, 3, 1, 3],
         [1, 1, 0, 0, 0, 1, 1],
         [1, 1, 1, 0, 3, 1, 0],
         [0, 1, 1, 2, 0, 1, 2

In [57]:
dataset.train_dataset[0]

{'program': tensor([ 0,  1,  2,  3,  4,  5,  5,  0,  6,  7,  8,  9,  5,  5, 10,  8, 11,  9,
         12, 13, 14, 15, 16,  8, 11,  9, 12, 13, 17, 15, 16,  8, 11,  9, 12, 13,
         18, 15, 19, 20, 21,  8, 14, 22, 14, 23, 17, 22, 17,  9, 12, 10,  8, 11,
         23, 18,  9, 12, 24, 24, 24, 24]),
 'choices': {'naming_scheme': (1, 'x'),
  'num_type': (1, 'double'),
  'print_function': (0, 'println'),
  'main_prompt': (0, 'Enter values to compute the Pythagorean Theorem.'),
  'input_prompt': (2, 'enter value '),
  'uses_method': (1, False),
  'output_prompt': (0, 'z=')}}

In [130]:
infer.model.forward(**batch)

KeyError: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')

In [128]:
torch.tensor([[0, 1], [2, 3]]).gather(dim=0, index=torch.tensor([0, 1]))

RuntimeError: invalid argument 4: Index tensor must have same dimensions as input tensor at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:440

In [ ]:
trainer = Trainer(dataset, ProgramClassifier, device=device)

In [ ]:
losses = []
for _ in tqdm(range(20)):
    losses.append(trainer.train_one_epoch())
pd.Series(losses).plot()

In [ ]:
trainer.eval()